In [3]:
global _debug
_debug = False

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

from gym_power_trading.envs import PowerTradingEnv
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback

#### Callback for logging rewards across parallel environments during Agent training

In [5]:
class VecRewardLogger(BaseCallback):
    def __init__(self, verbose=0):
        super(VecRewardLogger, self).__init__(verbose)
        self.rewards = []
        self.episode_rewards = None
    
    def _on_training_start(self):
        # Initialize on training start to get the number of environments
        self.episode_rewards = np.zeros(self.model.env.num_envs)

    def _on_step(self) -> bool:
        """ This method is called after each step """
        # Update rewards for all environments
        self.episode_rewards += np.array(self.locals['rewards'])
       
        # Check if any episode is done
        if np.any(self.locals['dones']):
            avg_episode_reward = np.mean(self.episode_rewards)
            self.rewards.append(avg_episode_reward)
            
            if self.verbose > 0:
                print("Logged rewards for completed episodes:", [self.rewards[-1]])
        return True  # Always return True to continue training

logger = VecRewardLogger(verbose=1)

In [6]:
file_name = "data\AEP_PSGC1_AMP_dart_Oct20.h5"
path = Path(file_name).resolve()
path_string = str(path)
df = pd.read_hdf(path_string)
df.head()

RT_LMP  DA_LMP
MARKET_DAY NODE          TYPE    VALUE HourEnding                
2020-10-01 AEP.PSGC1.AMP Gennode LMP   1            10.02   14.33
                                       2            12.51   13.09
                                       3            12.79   12.96
                                       4            12.53   13.62
                                       5            13.43   14.49

## PPO Agent with Basic Features 

### (4-hr Observation Window, 552,000 training iterations) 

Environment Observation Comprised of:
-  Current Price (\$/Mwh) / DA Price \($/Mwh)
-  Price Difference (from last tick) (%)
-  Battery State of Charge (%)
-  Battery Avg Energy Price (\$/Mwh) / Battery Avg Energy Price Window-period Rolling Avg \($/Mwh)



Rewards: 

    - Log Return of profitable discharges: Revenue / Power Sold * Stored Power Cost Basis
    - Log of revenue when selling power with negative cost basis: ln(revenue + |Stored Power Cost Basis|)

Penalty:

    - Overcharging (-1)
    - Discharging When Empty (-1)
    - Losing Money (-1)

In [7]:
%%time
window = 4 # hours
total_timesteps = 552_000
train_window = 240 # hrs 
n_envs = 4
validation_size = 3000
batch_timesteps = train_window * 1 * n_envs # 1 batch of train_window x window observations for each environment 

# Produce Observations across 16 parallel environments 
venv = make_vec_env(lambda: PowerTradingEnv(df=df, window_size=window, frame_bound=(0, train_window)), n_envs=n_envs) 
PPO_power_basic = PPO('MlpPolicy', venv, device='cpu')

# Walk forward during training to simulate real environment + reduce agent memorization
num_hours = (df.shape[0] - validation_size)
num_eps = num_hours // train_window
num_epochs = total_timesteps // (batch_timesteps * num_eps)

for epoch in range(num_epochs):
    for i in range(num_eps):
        start_day = i * train_window
        end_day = start_day + train_window 
        venv.env_method('set_frame_bound', start_day, end_day) # Set venvs frame bounds
        # Learn on currrent minibatch
        PPO_power_basic.learn(total_timesteps=batch_timesteps, callback=logger, reset_num_timesteps=False)

c:\users\andrew deur\documents\nyu\ds-ga 3001 reinforcement learning\3001-rl-grid-battery-management-agent\gym-power-trading\gym_power_trading\envs\power_trading.py:190: RuntimeWarning: divide by zero encountered in divide
  pct_diff = np.insert(np.where(prices[:-1] != 0, diff / prices[:-1], 0), 0, 0) # Change from price 1-tick ago
c:\Users\Andrew Deur\anaconda3\envs\rl_project\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.set_frame_bound to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.set_frame_bound` for environment variables or `env.get_wrapper_attr('set_frame_bound')` that will search the reminding wrappers.
  logger.warn(


Logged rewards for completed episodes: [-34.83367958152667]
Logged rewards for completed episodes: [-73.3456419975264]
Logged rewards for completed episodes: [-115.74419277027482]
Logged rewards for completed episodes: [-152.02488288287714]
Logged rewards for completed episodes: [-192.00224183771934]
Logged rewards for completed episodes: [-232.14587643595587]
Logged rewards for completed episodes: [-270.8087983328005]
Logged rewards for completed episodes: [-313.03195494068495]
Logged rewards for completed episodes: [-26.479669201813522]
Logged rewards for completed episodes: [-51.930359634701745]
Logged rewards for completed episodes: [-76.75146941393905]
Logged rewards for completed episodes: [-108.81105590514198]
Logged rewards for completed episodes: [-129.15030926572217]
Logged rewards for completed episodes: [-157.4098506650771]
Logged rewards for completed episodes: [-184.92014754691627]
Logged rewards for completed episodes: [-212.42382454589824]
Logged rewards for completed e

TypeError: cannot unpack non-iterable int object

### Episode Reward Progression

In [ ]:
num_eps = np.arange(1, len(logger.rewards) + 1)

plt.figure(figsize=(10, 5))
plt.plot(num_eps, logger.rewards)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('PPO Agent (Basic Features) 552,000 Iterations: Episode Reward Progression')
plt.grid(True)
plt.show()

### Plot of Agent Trades in Held out test set

In [ ]:
window = 3
last_index = len(df) - 10
start_index = last_index - validation_size
env = PowerTradingEnv(df=df, window_size=window, frame_bound=(start_index, last_index))
obs = env.reset()
obs = obs[0]

for i in range(last_index - start_index):
    action, states = PPO_power_basic.predict(obs, )
    obs, rewards, term, trunc, info = env.step(action)
    if term or trunc:
        history = env.history
        env.render_all(title="PPO Agent (Basic) Positions on Test Set @ 552,000 Training Steps", xlim=(1100, 1500))
        env.reset()

plt.plot()

### Reward Statistics over 100 Episodes

In [ ]:
mean_reward, reward_std = evaluate_policy(PPO_power_basic, env, n_eval_episodes=100)
print(f"Mean Reward:{ mean_reward:,.2f}\nReward std: {reward_std:,.2f}")

### Reward Evolution on Held Out Data

In [ ]:
plt.plot(history['total_reward'])
plt.title("PPO (Basic) Reward on Test Set @ 552,000 Training Steps")
plt.xlabel("Number of Timesteps (1hr)")
plt.show()

### Battery Charge Management

In [ ]:
plt.plot(history['battery_charge'])
plt.title("PPO (Basic) Battery Charge Management (552,000 Training Steps)")
plt.ylabel("Mwh")
plt.xlabel("Number of Timesteps (1hr)")
plt.show()

### Profit Evolution on Held Out Data

In [ ]:
plt.plot(history['total_profit'])
plt.title("PPO (Basic) Profit on Test Set @ 552,000 Training Steps")
plt.ylabel("$")
plt.xlabel("Number of Timesteps (1hr)")
plt.show()

In [ ]:
PPO_power_basic.save('PPO_power_basic_552k')

## PPO Agent with Advanced Features

### (4-hr Observation Window, 552,000 training iterations) 

Environment Observation Comprised of:
-  Current Price (\$/Mwh) 
-  DA Price \($/Mwh)
-  Battery State of Charge (%)
-  Battery Avg Energy Price (\$/Mwh) / Battery Avg Energy Price Window-period Rolling Avg \($/Mwh)

Rewards: 

    - Log Return of profitable discharges: Revenue / Power Sold * Stored Power Cost Basis
    - Log of revenue when selling power with negative cost basis: ln(revenue + |Stored Power Cost Basis|)

Penalty:

    - Overcharging (-1)
    - Discharging When Empty (-1)
    - Losing Money (-1)

In [ ]:
%%time
window = 4 # hours
total_timesteps = 552_000
train_window = 240 # hrs 
n_envs = 4
validation_size = 3000
batch_timesteps = train_window * 1 * n_envs # 1 batch of train_window x window observations for each environment 

# Produce Observations across 16 parallel environments 
venv2 = make_vec_env(lambda: PowerTradingEnv(df=df, window_size=window, frame_bound=(0, train_window)), n_envs=n_envs) 
PPO_power_adv = PPO('MlpPolicy', venv, device='cpu')
logger_adv = VecRewardLogger(verbose=1)

# Walk forward during training to simulate real environment + reduce agent memorization
num_hours = (df.shape[0] - validation_size)
num_eps = num_hours // train_window
num_epochs = total_timesteps // (batch_timesteps * num_eps)

for epoch in range(num_epochs):
    for i in range(num_eps):
        start_day = i * train_window
        end_day = start_day + train_window 
        venv2.env_method('set_frame_bound', start_day, end_day) # Set venvs frame bounds
        # Learn on currrent minibatch
        PPO_power_adv.learn(total_timesteps=batch_timesteps, callback=logger_adv, reset_num_timesteps=False)

### Episode Reward Progression

In [ ]:
num_eps = np.arange(1, len(logger_adv.rewards) + 1)

plt.figure(figsize=(10, 5))
plt.plot(num_eps, logger_adv.rewards)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('PPO Agent (Advanced Features) 552,000 Iterations: Episode Reward Progression')
plt.grid(True)
plt.show()

### Plot of Agent Trades in Held out test set

In [ ]:
last_index = len(df) - 10
start_index = last_index - 3000
env2 = PowerTradingEnv(df=df, window_size=window, frame_bound=(start_index, last_index))
obs = env2.reset()
obs = obs[0]

for i in range(last_index - start_index):
    action, states = PPO_power_adv.predict(obs)
    obs, rewards, term, trunc, info = env2.step(action)
    if term or trunc:
        history = env2.history
        pos = env2.render_all(title="PPO Agent (Advanced) Positions on Test Set @ 552,000 Training Steps", xlim=(1100, 1500))
        env2.reset()

plt.plot()

### Reward Statistics over 100 Episodes

In [ ]:
mean_reward, reward_std = evaluate_policy(PPO_power_adv, env2, n_eval_episodes=100)
print(f"Mean Reward:{ mean_reward:,.2f}\nReward std: {reward_std:,.2f}")

### Reward Evolution on Held Out Data

In [ ]:
plt.plot(history['total_reward'])
plt.title("PPO Agent (Advanced) Reward on Test Set @ 552,000 Training Steps")
plt.xlabel("Number of Timesteps (1hr)")
plt.show()

### Battery Charge Management

In [ ]:
### PPO 10,000 Steps on Sinusoid
plt.plot(history['battery_charge'])
plt.title("PPO Agent Battery Charge (Advanced) Reward on Test Set @ 552,000 Training Steps")
plt.ylabel("Mwh")
plt.xlabel("Number of Timesteps (1hr)")
plt.show()

### Profit Evolution on Held Out Data

In [ ]:
plt.plot(history['total_profit'])
plt.title("PPO Agent Profit (Advanced) Reward on Test Set @ 552,000 Training Steps")
plt.ylabel("$")
plt.xlabel("Number of Timesteps (1hr)")
plt.show()

In [ ]:
PPO_power_adv.save('PPO_power_advanced_552k')